In [3]:
import numpy as np
from scipy.stats import unitary_group
from functools import reduce
np.set_printoptions(linewidth=130)

U1 = unitary_group.rvs(4).reshape(2,2,2,2);
U2 = unitary_group.rvs(4).reshape(2,2,2,2);
U3 = unitary_group.rvs(4).reshape(2,2,2,2);

U1_ = U1.reshape(4,4).conj().T.reshape(2,2,2,2)
U2_ = U2.reshape(4,4).conj().T.reshape(2,2,2,2)
U3_ = U3.reshape(4,4).conj().T.reshape(2,2,2,2)

T1 = lambda U1, U2, U3: np.einsum(
    U1, [12,13,8,9],
    U1, [17,5,10,11],
    U2, [14,15,7,12],
    U2, [16,4,13,17],
    U3, [0,1,6,14],
    U3, [2,3,15,16],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)

T2 = lambda U1_, U2_, U3_: np.einsum(
    U3_, [0,12,6,7],
    U3_, [13,14,8,9],
    U2_, [1, 16,12,13],
    U2_, [17,15,14,10],
    U1_, [2,3,16,17],
    U1_, [4,5,15,11],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)

###########################################################
# Test that T1 and T2 are inverses of eachother
###########################################################


I = np.einsum(
    T1(U1,U2,U3), [0,1,2,3,4,5,12,13,14,15,16,17],
    T2(U1_,U2_,U3_), [12,13,14,15,16,17,6,7,8,9,10,11],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)

assert np.allclose(T1(U1,U2,U3).reshape(64,64) @ T2(U1_,U2_,U3_).reshape(64,64), np.eye(64))
assert np.allclose(I.reshape(64,64), np.eye(64))
############################################################
#
###########################################################
T3 = lambda T1, T2: np.einsum(
    T1, [0,1,2,3,4,5,12,13,16,17,18,19],
    T2, [14,15,16,17,18,19,6,7,8,9,10,11],
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
)

T4 = lambda U1,U2,U3,U1_,U2_,U3_: np.einsum(
    U3_, [10,12,4,5],
    U2_, [11,13,12,6],
    U1_, [14,15,13,7],
    U1,  [16,3,14,15],
    U2,  [17,2,9,16],
    U3,  [0,1,8,17],
    [0,1,2,3,4,5,6,7,8,9,10,11]
)
##################################
# test that T4 works as expcted
##################################


assert np.allclose(T4(np.eye(4).reshape(2,2,2,2), U2, U3, np.eye(4).reshape(2,2,2,2), U2_, U3_)[0,0,0,0,0,0,0,0,...].reshape(4,4), T4(U1, U2, U3, U1_, U2_, U3_)[0,0,0,0,0,0,0,0,...].reshape(4,4))
#####################################
#
#####################################

T5 = lambda T3, T4: np.einsum(
    T3, [2,3,4,5,6,7,10,11,12,13,14,15,21,20,23,22],
    T4, [0,1,21,20,8,9,23,22,16,17,18,19],
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
)

T5R = lambda T5: np.round(T5[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...].reshape(4,4),4)
T3R = lambda T3: np.round(T3[0,0,0,0,0,0,0,0,0,0,0,0,...].reshape(4,4),4)


U = [U1,U2,U3,U1_,U2_,U3_]

t1 = T1(*U[:3])
t2 = T2(*U[3:])
t3 = T3(t1,t2)
t4 = T4(*U)
t5 = T5(t3, t4)
t5r = T5R(t5)
t3r = T3R(t3)

assert(np.allclose(t5r, t3r))

def rfp(U): #right fixed point
    t1 = T1(*U[:3])
    t2 = T2(*U[3:])
    t3 = T3(t1,t2)
    t4 = T4(*U)
    t5 = T5(t3, t4)
    t5r = T5R(t5)
    t3r = T3R(t3)
    return t5r, t3r

t5r1, t3r1 = rfp(U)

assert np.allclose(t5r1, t3r1)



In [5]:
from scipy.linalg import expm

U1 = unitary_group.rvs(4).reshape(2,2,2,2)
U2 = unitary_group.rvs(4).reshape(2,2,2,2)
U3 = unitary_group.rvs(4).reshape(2,2,2,2)

M1 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B1 = 0.5 * (M1 - M1.conj().T)
M2 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B2 = 0.5 * (M2 - M2.conj().T)
M3 = np.random.rand(4,4) + 1j * np.random.rand(4,4)
B3 = 0.5 * (M3 - M3.conj().T)

U1_ = (expm(0.3 * B1)@ U1.reshape(4,4)).reshape(2,2,2,2)
U2_ = (expm(0.3 * B2)@ U2.reshape(4,4)).reshape(2,2,2,2)
U3_ = (expm(0.3 * B3)@ U3.reshape(4,4)).reshape(2,2,2,2)

U = [U1,U2,U3,U1_,U2_,U3_]

a, b = rfp(U)

In [6]:
print(a)

[[ 0.0055+0.0009j  0.0148-0.0021j -0.0148+0.0038j -0.0294-0.0218j]
 [ 0.0142+0.0159j -0.0073+0.0109j -0.0033+0.0131j  0.0001-0.0153j]
 [ 0.0243-0.0031j  0.0296+0.0218j  0.0028+0.0066j -0.0077-0.0055j]
 [-0.0269+0.0227j -0.0272+0.0059j -0.0114-0.0168j -0.0125-0.0014j]]


In [7]:
print(b)

[[-0.008 -0.0306j -0.014 -0.0685j -0.0881-0.0223j  0.0517-0.0129j]
 [ 0.0163-0.0251j  0.0191+0.05j    0.0419+0.0137j  0.0651+0.0201j]
 [-0.0101-0.0513j  0.0513-0.0055j -0.0403-0.0636j  0.1172+0.0327j]
 [ 0.0164+0.0428j -0.0852+0.0592j  0.0053+0.0471j -0.1092+0.0634j]]


In [11]:
U1 = unitary_group.rvs(4).reshape(2,2,2,2)
U2 = unitary_group.rvs(4).reshape(2,2,2,2)
U3 = unitary_group.rvs(4).reshape(2,2,2,2)

U1_ = unitary_group.rvs(4).reshape(2,2,2,2)
U2_ = unitary_group.rvs(4).reshape(2,2,2,2)
U3_ = unitary_group.rvs(4).reshape(2,2,2,2)

U = [U1,U2,U3,U1_,U2_,U3_]

a, b = rfp(U)

In [12]:
print(a)

[[ 0.0118+0.0288j -0.0197+0.0163j -0.0536+0.0098j -0.0186-0.0391j]
 [ 0.0013-0.0085j  0.0088-0.0073j  0.0161+0.0022j  0.0095+0.0032j]
 [-0.0128-0.0359j  0.0158-0.0201j  0.0601-0.011j   0.002 +0.0426j]
 [-0.0004-0.0103j -0.015 +0.0016j  0.0056-0.0035j -0.0553-0.0075j]]


In [14]:
a / b

array([[ 0.01095564-0.69603846j, -0.14925085-0.72517586j, -0.24591342-0.80079037j, -0.05642881-0.42507069j],
       [-0.14031047-0.59208813j, -0.10656842-0.40589218j, -0.47288379-0.49992826j, -0.0985881 -0.19644068j],
       [-0.0178064 -0.63341257j,  0.11276132-0.58280413j, -0.16934042-0.71446253j,  0.22827236-0.64381959j],
       [ 0.13209216-0.26843857j, -0.35057471-0.10791826j, -0.0539985 -0.08886927j, -1.60796165-0.77768444j]])